In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ns_f import ns_f
from optimal_traffic_scheduler import optimal_traffic_scheduler

First we initialize the data object, that stores information about all the packets that are and were transported in network. The index of this table is a unique identifier of the packet. The optional argument sets the inital size of that table.

In [2]:
dat = ns_f.data(1000)

In the next step we define the network by creating a number of nodes (servers and clients) with certain properties:

In [3]:
setup_dict_server = {}
setup_dict_server['v_max'] = 2000  # packets / s
setup_dict_server['s_max'] = 200  # packets
setup_dict_server['timeout'] = 1  # s

In [4]:
input_1 = ns_f.server(setup_dict_server, dat, name='input_1')
input_2 = ns_f.server(setup_dict_server, dat, name='input_2')
output_1 = ns_f.server(setup_dict_server, dat, name='output_1')
output_2 = ns_f.server(setup_dict_server, dat, name='output_2')
server_1 = ns_f.server(setup_dict_server, dat, name='server_1')
server_2 = ns_f.server(setup_dict_server, dat, name='server_2')

Furthermore we create connections between these nodes by defining a number of circuits. Each circuit has a start an end and an arbitrary number of stops in between. We create a very simple network with two circuits:

In [5]:
circuits = [
    {'route': [input_1, server_1, server_2, output_1]},
    {'route': [input_2, server_1, server_2, output_2]},
]

We can now call:

In [6]:
nw = ns_f.network(data=dat)
nw.from_circuits(circuits)

The network is defined now and can be simulated with TCP control, if desired. However, we want to use the OTS control algorithm. This distributed control algorithm runs independently at every server node and can be setup as follows:

In [7]:
dt_ots = 0.1
N_steps = 20
ots_weights = {'control_delta': 0.1, 'send': 1, 'store': 0, 'receive': 1}

In [8]:
# OTS clients
input_1.set_ots_client(dt_ots, N_steps)
input_2.set_ots_client(dt_ots, N_steps)
output_1.set_ots_client(dt_ots, N_steps)
output_2.set_ots_client(dt_ots, N_steps)

# OTS 
server_1.set_ots(dt_ots, N_steps, ots_weights)
server_2.set_ots(dt_ots, N_steps, ots_weights)

We differentiate between OTS nodes and ots clients. OTS nodes run an optimization problem to determine optimal sending rates and allotments for incoming packets, whereas clients will use simple operations to determine future sending rates and thresholds for incoming packet streams. 

The last step of the process involves calling:

In [9]:
nw.setup_ots(dt_ots, N_steps)

The optimizers are now ready to run.

We add some packets to the inputs, such that the network has something to do:

In [10]:
input_1.add_2_buffer(buffer_ind=0, circuit=0, n_packets=1000)
input_2.add_2_buffer(buffer_ind=0, circuit=1, n_packets=2000)

In [11]:
nw.run_ots()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       80
Number of nonzeros in inequality constraint Jacobian.:     1520
Number of nonzeros in Lagrangian Hessian.............:      140

Total number of variables............................:      100
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       40
Total number of inequality constra

In [12]:
def get_con_props(nw):
    con = nw.connections.copy()
    #We query these attributes from the connections object ('prop') that is part of the dataframe
    con['max_window_size'] = con.apply(lambda row: row.prop.window_size, axis=1)
    con['window_size'] = con.apply(lambda row: len(row.prop.window), axis=1)
    con['transit_size'] = con.apply(lambda row: len(row.prop.transit), axis=1)
    con['trans_reply_size'] = con.apply(lambda row: len(row.prop.transit_reply), axis=1)
    
    con['v_con'] = con.apply(lambda row: np.round(row.v_con[0],1),axis=1)
    con['c_con'] = con.apply(lambda row: np.round(row.c_con[0],2),axis=1)
    con['v_max'] = con.apply(lambda row: np.round(row.v_max[0],1),axis=1)
    con['bwl_t'] = con.apply(lambda row: np.round(row.bandwidth_load_target[0], 2),axis=1)
    con['ml_t'] = con.apply(lambda row: np.round(row.memory_load_target[0],2),axis=1)
    con['bwl_s'] = con.apply(lambda row: np.round(row.bandwidth_load_source[0],2),axis=1)
    con['ml_s'] = con.apply(lambda row: np.round(row.memory_load_source[0],2),axis=1)
    
      
    # And display only the relevant columns:
    return con[['source_name','target_name','circuit','max_window_size','window_size','transit_size','trans_reply_size','v_con','c_con','v_max','bwl_t','ml_t','bwl_s', 'ml_s']]
    

In [119]:
if nw.t_next_iter <= nw.t:
    nw.run_ots()
else:
    nw.simulate()
print(nw.t)

0.5100000000000002


In [120]:
get_con_props(nw)

,source_name,target_name,circuit,max_window_size,window_size,transit_size,trans_reply_size,v_con,c_con,v_max,bwl_t,ml_t,bwl_s,ml_s
0,input_1,server_1,[0],250,106,34,35,[[491.4]],[[1]],[[493.9]],[[0.97]],[[1.0]],[[0.0]],[[0.0]]
1,server_1,server_2,"[0, 1]",291,136,69,67,[[948.6]],"[[0.49], [0.51]]",[[1266.9]],[[0.88]],[[0.73]],[[0.97]],[[1.0]]
2,server_2,output_1,[0],96,66,34,32,[[456.7]],[[1.0]],[[2000.0]],[[0.0]],[[0.0]],[[0.88]],[[0.73]]
3,input_2,server_1,[1],250,109,34,28,[[491.4]],[[1]],[[493.9]],[[0.97]],[[1.0]],[[0.0]],[[0.0]]
4,server_2,output_2,[1],94,64,32,32,[[276.5]],[[1.0]],[[2000.0]],[[0.0]],[[0.0]],[[0.88]],[[0.73]]


In [116]:
nw.connections.loc[1].v_con[0]

array([[1017.20632206]])

In [125]:
nw.nodes.loc[0].node.transit

AttributeError: 'server' object has no attribute 'transit'